In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from animate import *
import sys
sys.path.append('../../src/')
from helper_functions import *
plt.style.use('dynamics_site')

In [3]:
#define findiff approx:
def getsol(tmax,f0,dt):
    y=[f0]
    x=[0]
    time=0
    while time < tmax:
        f_next = f0/(1+ 15*dt)
        y.append(f0)
        x.append(time)
        f0 = f_next
        time= time+dt
    return x, y


def generator(dt):
    matplotlib.use('agg')
    # Set up data
    x = np.linspace(0, 2, 100)
    y = np.e**(-15*x)
    newx,newy = getsol(2,1,dt)
    
    fig,ax = plt.subplots(figsize=(7,7))
    plt.plot(x,y,c='k',label='$e^{t}$',zorder=0)
    plt.scatter(newx,newy,c='r',label='numerical f(t)',zorder=1)
    plt.ylabel("f(t)")
    plt.xlabel('t')
    plt.title('$\Delta t =$'+str(np.round(dt,3)))
    plt.legend(loc='lower right')
    plt.ylim(-0.5,1.25)
    plt.close() #stops figure from being shown
    return fig

In [4]:
parameter_grid = np.linspace(0.25,0.001,150)
savefigures(generator,parameter_grid,outdir='./inplicit1/')
render('./inplicit1/','inplicit1','gif',cleanup_type='rm')

saving figures


100%|█████████████████████████████████████████| 150/150 [00:29<00:00,  5.05it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora 

Framerate is None fps
Total Runtime is None s


[Parsed_palettegen_0 @ 0x5569c92c9000] 255(+1) colors generated out of 1516 colors; ratio=0.168206
[Parsed_palettegen_0 @ 0x5569c92c9000] Dupped color: FFD90000
[Parsed_palettegen_0 @ 0x5569c92c9000] Dupped color: FFFF3737
Output #0, image2, to '../temp_palette_inplicit1.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc58.134.100 png
frame=    1 fps=0.6 q=-0.0 Lsize=N/A time=00:00:00.04 bitrate=N/A speed=0.0244x    
video:1kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enabl

## Define Integration Loop

In [5]:
def integrate_euler(system,dt,steps):
    G, mvec, rvec, vvec = load_initials(system)
    Rs = [rvec]
    Vs = [vvec]

    for i in range(steps-1):
        #do fixed point iteration
        trial_new_vvec_in = vvec
        trial_new_rvec_in = rvec
        err = np.inf 
        while err > 1e-12:
            trial_new_vvec_out = vvec + dt*get_acclerations(rvec + dt*trial_new_vvec_in,mvec)
            trial_new_rvec_out = rvec + dt*(vvec + dt*get_acclerations(trial_new_rvec_in,mvec))
            err = np.max(np.abs([trial_new_vvec_in-trial_new_vvec_out,trial_new_rvec_in - trial_new_rvec_out]))
            trial_new_vvec_in = trial_new_vvec_out
            trial_new_rvec_in = trial_new_rvec_out
        #this converges to new_vvec and new_rvec in 3-4 iterations
        new_rvec = trial_new_rvec_in
        new_vvec = trial_new_vvec_in
        Rs.append(new_rvec)
        Vs.append(new_vvec)
        rvec = new_rvec
        vvec = new_vvec

    Rs = np.array(Rs)
    Vs = np.array(Vs)
    PE = get_PE(Rs,mvec)
    KE = get_KE(Vs,mvec)
    
    Rs = np.array([Rs[j].T for j in range(len(Rs))])
    Vs = np.array([Vs[j].T for j in range(len(Rs))])
    return Rs,Vs,PE+KE

## Preform Integration

In [6]:
dt = 0.001
steps = 4000
Rs,Vs,Es = integrate_euler('../../src/Earth-Sun.npy',dt,steps)
xs,ys = Rs[:,:2,0].T
xp,yp = Rs[:,:2,1].T
times = np.linspace(0,steps*dt,steps)

## Generate Output 

In [7]:
def makeplot(i):
    xs,ys = Rs[:i,:2,0].T
    xp,yp = Rs[:i,:2,1].T
    relative_error = Es[:i]
    times = np.linspace(0,steps*dt,steps)


    fig,ax = plt.subplots(ncols=2,figsize=(20,10))
    ax[0].scatter(xs[-1],ys[-1],c='k',marker='*',s=100)
    ax[0].plot(xp,yp,c='k',alpha=0.3)
    ax[0].scatter(xp[-1],yp[-1],c='k',s=100)
    ax[0].set_xlabel("X [AU]")
    ax[0].set_ylabel("Y [AU]")
    ax[0].set_xlim(-1.25*np.max(Rs),1.25*np.max(Rs))
    ax[0].set_ylim(-1.25*np.max(Rs),1.25*np.max(Rs))

    ax[1].plot(times[:i],relative_error,c='k')
    ax[1].set_ylabel("Energy(t)")
    ax[1].set_xlabel("Time [yrs]")
    ax[1].set_xlim(0,np.max(times))
    ax[1].set_ylim(np.min(Es),np.max(Es))
    plt.suptitle(r"$\Delta t = 0.001 [yrs]$")
    return fig

In [8]:
parameter_grid = list(range(1,len(Rs),50)) #plot every 5th timestep
savefigures(makeplot,parameter_grid,outdir='./inplicit2/')
render('./inplicit2/','inplicit2','gif',cleanup_type='rm')

saving figures


100%|███████████████████████████████████████████| 80/80 [00:42<00:00,  1.88it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora 

Framerate is None fps
Total Runtime is None s


[Parsed_palettegen_0 @ 0x55a86f58eb80] 255(+1) colors generated out of 256 colors; ratio=0.996094
Output #0, image2, to '../temp_palette_inplicit2.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc58.134.100 png
frame=    1 fps=0.2 q=-0.0 Lsize=N/A time=00:00:00.04 bitrate=N/A speed=0.00985x    
video:1kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2